In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
df = pd.read_excel('data/Muesli Project raw data - group 3.xlsx', header=1)
df1 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=1)
df2 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=2)
df3 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=3)

In [40]:
print(df.columns)
print(df1.columns)
print(df2.columns)
print(df3.columns)

Index(['Index', 'Order ID', 'Order Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Origin Channel', 'Country/Region', 'City', 'State',
       'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product ID',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date', 'Customer Name'], dtype='object')
Index(['Row ID', 'Order ID', 'Order Date', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


In [42]:
print(df.shape)
print(df1.shape)
print(df2.shape)
print(df3.shape)

(9994, 19)
(333, 3)
(5899, 5)
(290, 3)
